<a href="https://colab.research.google.com/github/Mohammed-Taasir/IR_2023-GROUP-14/blob/assignment-1/IR_Assignment_Question_3(A).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/IR_Assignment_1_Dataset/CSE508_Winter2023_Dataset.zip' '/content/'
!unzip 'CSE508_Winter2023_Dataset.zip' &> /dev/null
!rm 'CSE508_Winter2023_Dataset.zip'

In [ ]:
path = '/content/CSE508_Winter2023_Dataset/'

In [ ]:
import glob
import pandas as pd
import numpy as np
import string
import os
import re
import pickle
import nltk

In [ ]:
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader 
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# * Defining Helper Functions

### Read from files

In [ ]:
def getListOfFiles(directory):

    # Parameters: directory: type(string)        
    # returns: list of all files in directory with the full path of file
    
    list_of_files = []

    for i in range(len(filenames)):
      fi = "/content/CSE508_Winter2023_Dataset/"+filenames[i]
      list_of_files.append(fi)
    
    return list_of_files

### Preprocessing Functions

In [ ]:
def lowercase(data):

    # Parameters: data: type(string)
    # returns: lowercase of data   
     
    return data.lower()

In [ ]:
def perform_word_tokenize(corpus):
  
    # Parameters:corpus: type(string)   
    # returns word-level tokenization of corpus

    return word_tokenize(corpus)

In [ ]:
def remove_stopwords_from_tokens(tokens, stopwords_set):
  
    # Parameters: tokens: type(list)
    #             stopwords_set: type(set)
    # returns: tokens without stopwords

    tokens_sans_stopwords = [x for x in tokens if x not in stopwords_set] 
    return tokens_sans_stopwords

In [ ]:
def remove_punctuation_from_tokens(tokens):

    # Parameters: tokens: type(list)
    # returns: tokens without punctuation

    tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in tokens]
    return tokens_sans_punctuation

In [ ]:
def remove_blank_space_tokens(tokens):
    
    #Parameters: tokens: type(list)
    #returns: tokens without blank tokens

    tokens_sans_blank_space = [x for x in tokens if x!='']  
    return tokens_sans_blank_space

In [ ]:
def preprocess(corpus, stopwords_set, preprocess_type):
    # Convert the text to lower case
    lowercase_corpus = lowercase(corpus)
    
    # Perform word tokenization (word_tokenize also takes care of whitespace)
    word_tokens = perform_word_tokenize(lowercase_corpus)
    
    # Remove stopwords from tokens
    word_tokens_sans_stopwords = remove_stopwords_from_tokens(word_tokens, stopwords_set)
    
    # Remove punctuation marks from tokens
    word_tokens_sans_punctuation = remove_punctuation_from_tokens(word_tokens_sans_stopwords)
    
    # Remove blank space tokens
    word_tokens_sans_blank_tokens = remove_blank_space_tokens(word_tokens_sans_punctuation)
  
    
    if preprocess_type=='query':
        return word_tokens_sans_blank_tokens                              # if its a query return tokens in the same order
    return sorted(list(dict.fromkeys(word_tokens_sans_blank_tokens)))     # if its a whole document then return tokens in sorted manner

In [ ]:
def create_file_dictionary(list_of_files):
    
    # Paramteres: list_of_files: type(string)
    # returns: file_dictionary with integer key and path_of_file as value
  
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

# Q1. Data Preprocessing

## (i) Relevant Text Extraction

In [ ]:
corpus_root='/content/CSE508_Winter2023_Dataset/'
corpus=PlaintextCorpusReader(corpus_root,'.*')

In [ ]:
# Printing names of all files

filenames=corpus.fileids()
print(filenames)

['cranfield0001', 'cranfield0002', 'cranfield0003', 'cranfield0004', 'cranfield0005', 'cranfield0006', 'cranfield0007', 'cranfield0008', 'cranfield0009', 'cranfield0010', 'cranfield0011', 'cranfield0012', 'cranfield0013', 'cranfield0014', 'cranfield0015', 'cranfield0016', 'cranfield0017', 'cranfield0018', 'cranfield0019', 'cranfield0020', 'cranfield0021', 'cranfield0022', 'cranfield0023', 'cranfield0024', 'cranfield0025', 'cranfield0026', 'cranfield0027', 'cranfield0028', 'cranfield0029', 'cranfield0030', 'cranfield0031', 'cranfield0032', 'cranfield0033', 'cranfield0034', 'cranfield0035', 'cranfield0036', 'cranfield0037', 'cranfield0038', 'cranfield0039', 'cranfield0040', 'cranfield0041', 'cranfield0042', 'cranfield0043', 'cranfield0044', 'cranfield0045', 'cranfield0046', 'cranfield0047', 'cranfield0048', 'cranfield0049', 'cranfield0050', 'cranfield0051', 'cranfield0052', 'cranfield0053', 'cranfield0054', 'cranfield0055', 'cranfield0056', 'cranfield0057', 'cranfield0058', 'cranfield005

In [ ]:
print(type(filenames))
print(len(filenames))

<class 'list'>
1400


## First 5 Documents

In [ ]:
# Printing contents of first 5 files before extracting

for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------
<DOC>
<DOCNO>
1
</DOCNO>
<TITLE>
experimental investigation of the aerodynamics of a
wing in a slipstream .
</TITLE>
<AUTHOR>
brenckman,m.
</AUTHOR>
<BIBLIO>
j. ae. scs. 25, 1958, 324.
</BIBLIO>
<TEXT>
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the des

## Extracting content between TITLE and TEXT tag from all documents.

In [ ]:
for i in range(len(filenames)):
    outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
    doc = outfile.read()

    # initializing string
    test_str = doc

    # initializing tags
    tag1 = "TITLE"
    tag2 = "TEXT"

    # regex to extract required strings
    reg_str1 = "<"+tag1+">(.*?)</"+tag1+">"
    res1 = re.findall(reg_str1, test_str, re.DOTALL)

    reg_str2 = "<"+tag2+">(.*?)</"+tag2+">"
    res2 = re.findall(reg_str2, test_str, re.DOTALL)

    #Combining contents of TITLE and TEXT
    res = res1+res2
    
    s = res

    # using list comprehension
    listToStr = ' '.join([str(elem) for elem in s])

    writeFile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'w')
    L = listToStr

    writeFile.write(L)
    writeFile.close()

    outfile.close()

## First 5 Documents after extracting contents between TITLE and TEXT tag.

In [ ]:
for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-----------------------------------------------

# (ii) Preprocessing

## 1. Lowercase the text

In [ ]:
newfiles=[]
for i in range(len(filenames)):
  fileind=open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  filedata=fileind.read()
  filedata=lowercase(filedata)
  newfiles.append(filedata)
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-------------------------------------------------

## 2. Perform tokenization

In [ ]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  filedata=perform_word_tokenize(filedata)
  newfiles[i] = filedata
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'of', 'the', 'aerodynamics', 'of', 'a', 'wing', 'in', 'a', 'slipstream', '.', 'an', 'experimental', 'study', 'of', 'a', 'wing', 'in', 'a', 'propeller', 'slipstream', 'was', 'made', 'in', 'order', 'to', 'determine', 'the', 'spanwise', 'distribution', 'of', 'the', 'lift', 'increase', 'due', 'to', 'slipstream', 'at', 'different', 'angles', 'of', 'attack', 'of', 'the', 'wing', 'and', 'at', 'different', 'free', 'stream', 'to', 'slipstream', 'velocity', 'ratios', '.', 'the', 'results', 'were', 'intended', 'in', 'part', 'as', 'an', 'evaluation', 'basis', 'for', 'different', 'theoretical', 'treatments', 'of', 'this', 'problem', '.', 'the', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'with', 'supporting', 'evidence', ',', 'showed', 'that', 'a', 'substantial', 'part', 'of', 'the', 'lift', 'increment', 'produced', 'by', 'the', 'slipstream', 'was', 'due', 'to', 'a', '/destalling/', 'or', 'boundary-layer-control'

In [ ]:
stopwords_set = set(stopwords.words('english'))

## 3. Remove stopwords

In [ ]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_stopwords = remove_stopwords_from_tokens(filedata, stopwords_set)
  newfiles[i] = tokens_sans_stopwords
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '.', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '.', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '.', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'supporting', 'evidence', ',', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', '/destalling/', 'boundary-layer-control', 'effect', '.', 'integrated', 'remaining', 'lift', 'increment', ',', 'subtracting', 'destalling', 'lift', ',', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '.', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '.']
--

## 4. Remove punctuations

In [ ]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_punctuation = remove_punctuation_from_tokens(filedata)
  newfiles[i] = tokens_sans_punctuation
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '', 'comparative', 'span', 'loading', 'curves', '', 'together', 'supporting', 'evidence', '', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', '', 'integrated', 'remaining', 'lift', 'increment', '', 'subtracting', 'destalling', 'lift', '', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '']
----------------

## 5. Remove blank space tokens

In [ ]:
lent = 0
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_blank_space = remove_blank_space_tokens(filedata)
  newfiles[i] = tokens_sans_blank_space
  lent += len(newfiles[i])
print(len(newfiles))
print(lent)

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
127377
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', 'comparative', 'span', 'loading', 'curves', 'together', 'supporting', 'evidence', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', 'integrated', 'remaining', 'lift', 'increment', 'subtracting', 'destalling', 'lift', 'found', 'agree', 'well', 'potential', 'flow', 'theory', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment']
-------------------------------------------------

In [ ]:
fileData = newfiles

# QUESTION 2 STARTS HERE


# Q2. Boolean Queries

### 1. Create a unigram inverted index(from scratch; No library allowed) of the dataset obtained from Q1 (after preprocessing).

### 2. Use Python’s pickle module to save and load the unigram inverted index.

In [ ]:
def create_unigram_inverted_index(file_dictionary, stopwords_set):
    # initialize unigram inverted index
    unigram_inverted_index = {}
    
    # unigram inverted index
    for doc_ID in range(len(file_dictionary)):
        file = open(file_dictionary[doc_ID], 'r', encoding='utf-8', errors='ignore')
        file_corpus = file.read()
        file.close()
        doc_tokens = preprocess(file_corpus, stopwords_set, 'doc')
        for token in doc_tokens:
            if token in unigram_inverted_index:
                unigram_inverted_index[token][0] += 1               # stores doc_frequency
                unigram_inverted_index[token][1].append(doc_ID)     # appending docs in which that term occurs
            else:
                unigram_inverted_index[token] = []
                unigram_inverted_index[token].append(1)
                unigram_inverted_index[token].append([doc_ID])      # initializing if this token appears for first time
    
    # Saving unigram inverted index using pickle module  
    uii_file = open('unigram_inverted_index_pickle_file', 'wb')
    pickle.dump(unigram_inverted_index, uii_file)
    uii_file.close()

    return unigram_inverted_index

stopwords_set = set(stopwords.words('english'))

list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset/')
uni = create_unigram_inverted_index(list_of_files,stopwords_set)

In [ ]:
uii_file = open('unigram_inverted_index_pickle_file', 'rb')
unigram_inverted_index = pickle.load(uii_file)
uii_file.close()

In [ ]:
print(len(unigram_inverted_index))

8996


### 3. Provide support for the following operations: 
a. T1 AND T2<br>
b. T1 AND NOT T2<br>
c. T1 OR T2<br>
d. T1 OR NOT T2

AND

In [ ]:
def xANDy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            i+=1
        else:
            j+=1
        num_of_operations+=1
    
    return result, num_of_operations

OR

In [ ]:
def xORy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
        else:
            result.add(pos_list_2[j])
            j+=1
        num_of_operations+=1
    
    while(i<len(pos_list_1)):
        result.add(pos_list_1[i])
        i+=1
    while(j<len(pos_list_2)):
        result.add(pos_list_2[j])
        j+=1
    
    return result, num_of_operations

AND NOT

In [ ]:
def xANDNOTy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
        else:
            j+=1
        num_of_operations+=1
    
    while(i<len(pos_list_1)):
        result.add(pos_list_1[i])
        i+=1
    
    return result, num_of_operations

OR NOT

In [ ]:
def xORNOTy(pos_list_1, pos_list_2, u_list):
    num_of_not_operations = 0
    i,j = 0,0
    not_pos_list_2 = SortedSet()
    while(i<len(pos_list_2) and j<len(u_list)):
        if pos_list_2[i]<u_list[j]:
            i+=1
        elif pos_list_2[i]>u_list[j]:
            not_pos_list_2.add(u_list[j])
            j+=1
        else:
            i+=1
            j+=1
        num_of_not_operations+=1
    while(j<len(u_list)):
        not_pos_list_2.add(u_list[j])
        j+=1
    
    result, ops = xORy(pos_list_1, not_pos_list_2)
    return result, num_of_not_operations+ops

### 4. Queries should be generalized i.e., you should provide support for queries like T1 AND T2 OR T3 OR T4 ....

### 5. You are also required to compute the minimum number of comparisons done to execute the query [only where merging is required]

### 6. Input format: <br>
a. The first line contains N denoting the number of queries to execute<br>
b. The next 2N lines contain queries in the following format:<br>
&ensp; i. Input sequence<br>
&ensp; ii. Operations separated by comma

### 7. Output Format:
a. 4N lines consisting of the results in the following format:<br>
&ensp;i. Query X<br>
&ensp;ii. Number of documents retrieved for query X<br>
&ensp;iii. Names of the documents retrieved for query X<br>
&ensp;iv. Number of comparisons required for query X<br>

### 8. Perform preprocessing steps (from Q1) on the input sequence as well.

In [ ]:
def main():
    # create set of stop words for preprocessing
    stopwords_set = set(stopwords.words('english'))
    
    # Get List of Files in Dataset
    list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset/')
    
    # create dictionary of file with docID (integer) as key and full_path of file as value
    file_dictionary = create_file_dictionary(list_of_files)
    
    # create unigram inverted index once and then load pickle file afterwards
    create_unigram_inverted_index(file_dictionary, stopwords_set)
    
    #Loading pre-processed files
    uii_file = open('unigram_inverted_index_pickle_file', 'rb')
    unigram_inverted_index = pickle.load(uii_file)
    uii_file.close()
    u_list = SortedSet()
    for a in range(len(file_dictionary)):
        u_list.add(a)
    
    N = int(input())                                                            # N denoting the number of queries to execute
    for q in range(N):
        input_sentence = input()                                                # Taking input query
        input_operation_sequence = input()                                      # Taking operations to be done as an input 
        sanitized_query = preprocess(input_sentence, stopwords_set, 'query')    # Preprocessing the input query
        sanitized_operation_sequence = []                             
        for op_seq in input_operation_sequence.split(','):
            sanitized_operation_sequence.append(op_seq.lower().strip())         
        print(sanitized_query)                                                  # Printing Input sequence
        print(sanitized_operation_sequence)                                     # Printing operations seperated by comma
        if(len(sanitized_query)!=len(sanitized_operation_sequence)+1):          
            print('invalid query')
            continue    # will continue with next query 
        ptr1,ptr2 = 0,0
        result = unigram_inverted_index[sanitized_query[ptr1]][1]
        num_of_operations = 0
        while(ptr2<len(sanitized_operation_sequence)):                          # Processing of the input query X starts here.
            if sanitized_operation_sequence[ptr2]=='and':
                res, ops = xANDy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='or':
                res, ops = xORy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='and not':
                res, ops = xANDNOTy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='or not':
                res, ops = xORNOTy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1], u_list)
            ptr1+=1
            ptr2+=1
            result = res
            num_of_operations += ops
        print('Number of documents matched: {}'.format(len(result)))            # Printing Number of documents retrived for query X.
        print('Number of comparisons required: {}'.format(num_of_operations))   # Printing Number of comparisons required for query X.
        print('Documents: \n')                                                  # Printing Name of Documents retrived for query X.
        for res_doc in result:
            print(file_dictionary[res_doc])

In [ ]:
if __name__ == "__main__":
    main()

1
transition supersonic wind
and, and
['transition', 'supersonic', 'wind']
['and', 'and']
Number of documents matched: 7
Number of comparisons required: 434
Documents: 

/content/CSE508_Winter2023_Dataset/cranfield0007
/content/CSE508_Winter2023_Dataset/cranfield0040
/content/CSE508_Winter2023_Dataset/cranfield0171
/content/CSE508_Winter2023_Dataset/cranfield0182
/content/CSE508_Winter2023_Dataset/cranfield0272
/content/CSE508_Winter2023_Dataset/cranfield0710
/content/CSE508_Winter2023_Dataset/cranfield1211


In [ ]:
for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

----------
File : 1
----------
['experimental investigation', 'investigation aerodynamics', 'aerodynamics wing', 'wing slipstream', 'slipstream experimental', 'experimental study', 'study wing', 'wing propeller', 'propeller slipstream', 'slipstream made', 'made order', 'order determine', 'determine spanwise', 'spanwise distribution', 'distribution lift', 'lift increase', 'increase due', 'due slipstream', 'slipstream different', 'different angles', 'angles attack', 'attack wing', 'wing different', 'different free', 'free stream', 'stream slipstream', 'slipstream velocity', 'velocity ratios', 'ratios results', 'results intended', 'intended part', 'part evaluation', 'evaluation basis', 'basis different', 'different theoretical', 'theoretical treatments', 'treatments problem', 'problem comparative', 'comparative span', 'span loading', 'loading curves', 'curves together', 'together supporting', 'supporting evidence', 'evidence showed', 'showed substantial', 'substantial part', 'part lift', 

In [ ]:
newlist=[]
newlist=newfiles
for i in range(len(newfiles)):
   for j in range(len(newfiles[i])-1):
       temp1=newfiles[i][j]
       temp2=newfiles[i][j+1]
       temp=temp1+' '+temp2
       newlist[i][j]=temp

   newlist[i].pop()
bigram_inverted_index = {}
for i in range(len(newlist)):
        # file = open(file_dictionary[doc_ID], 'r', encoding='utf-8', errors='ignore')
        # file_corpus = file.read()
        # file.close()
        # doc_tokens = preprocess(file_corpus, stopwords_set, 'doc')
    doc_tokens = newlist[i]
    for token in doc_tokens:
        if token in bigram_inverted_index:
            bigram_inverted_index[token][0] += 1   # stores doc_frequency
            if( i not in bigram_inverted_index[token][1]):
              bigram_inverted_index[token][1].append(i)    # appending docs in which that term occurs
              
        else:
            bigram_inverted_index[token] = []
            bigram_inverted_index[token].append(1)
            bigram_inverted_index[token].append([i])    # initializing if this token appears for first time'
pi_file = open('bigram_inverted_index_pickle_file', 'wb')
pickle.dump(bigram_inverted_index, pi_file)
pi_file.close()

In [ ]:
bigram_file = open('bigram_inverted_index_pickle_file', 'rb')
bigram_inverted_index = pickle.load(bigram_file)
bigram_file.close()

In [ ]:
uii_file = open('unigram_inverted_index_pickle_file', 'rb')
unigram_inverted_index = pickle.load(uii_file)
uii_file.close()

In [ ]:
b=search_bigram_query()

1
search your phrase queriesexperimental investigation
Number of documents retrieved for query 1 using positional inverted index:50
Number of documents retrieved for query 1 using positional inverted index:
/content/CSE508_Winter2023_Dataset/cranfield0001
/content/CSE508_Winter2023_Dataset/cranfield0029
/content/CSE508_Winter2023_Dataset/cranfield0084
/content/CSE508_Winter2023_Dataset/cranfield0170
/content/CSE508_Winter2023_Dataset/cranfield0173
/content/CSE508_Winter2023_Dataset/cranfield0176
/content/CSE508_Winter2023_Dataset/cranfield0189
/content/CSE508_Winter2023_Dataset/cranfield0245
/content/CSE508_Winter2023_Dataset/cranfield0251
/content/CSE508_Winter2023_Dataset/cranfield0372
/content/CSE508_Winter2023_Dataset/cranfield0423
/content/CSE508_Winter2023_Dataset/cranfield0442
/content/CSE508_Winter2023_Dataset/cranfield0497
/content/CSE508_Winter2023_Dataset/cranfield0505
/content/CSE508_Winter2023_Dataset/cranfield0522
/content/CSE508_Winter2023_Dataset/cranfield0552
/content/

**Q3.(ii) Positional Indexing**

In [ ]:
len(newfiles)

1400

In [ ]:
import os
import pickle

def create_positional_index(newfiles):
    positional_index = {}
    for i in range(len(newfiles)):
        tokens = newfiles[i]
        for j, word in enumerate(tokens):
            if(word in positional_index):
                if(i in positional_index[word]):
                    positional_index[word][i].append(j)
                else:
                    positional_index[word][i] = [j]
            else:
                positional_index[word] = {i:[j]}
    
    pi_file = open('positional_index_pickle_file', 'wb')
    pickle.dump(positional_index, pi_file)
    pi_file.close()

    return positional_index

In [ ]:
positional_index = create_positional_index(newfiles)
pi_file = open('positional_index_pickle_file', 'rb')
positional_index = pickle.load(pi_file)
pi_file.close()

In [ ]:
def intersection(lst1, lst2):
 
    # Use of hybrid method
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

In [ ]:
def difference_within_k(pos1,pos2,k):
  for i in pos1:
    for j in pos2:
      if( i<j and abs(j - i) == k):
        return 1;
  return 0;

In [ ]:
def search_bigram(query,q):
          res = len(query.split())
          if(res>5):
            print(" bigram length of the input query to be <= 5")
            return
            
            
          sanitized_query = preprocess(query, stopwords_set, 'query')

          flag = 0
          if(len(sanitized_query) == 0):
            print("bigram invalid query")
            return

          if(len(sanitized_query) == 1):
            print(" bigram invalid query")
            return

          flag = 0
          for token in sanitized_query:
            if(token not in unigram_inverted_index):
              print(" bigram invalid query")
              return 
        

          result = []
          for i in range(len(sanitized_query)-1):
            biword = str(sanitized_query[i] + " " + sanitized_query[i+1])
            if i==0:
              result = bigram_inverted_index[biword][1]
            else:
              result = result.intersection(bigram_inverted_index[biword][1])
          
          if(len(result)  == 0):
            print(" bigram no document found")
            return
            

          print('Number of documents retrieved for query {} using bigram inverted index: {}'.format(q,len(result)))
          print('Number of documents retrieved for query {} using bigram inverted index:'.format(q))
          for res_doc in result:
            print(list_of_files[res_doc])

In [ ]:
def search_positional(query,q):
          res = len(query.split())
          if(res>5):
              print("positional length of the input query to be <= 5")
              return
          
          afterPreprocessQuery = preprocess(query, stopwords_set, 'query')
          #print(afterPreprocessQuery)
         
          if(len(afterPreprocessQuery) == 0):
            print(" positional invalid query")
            return
        

          pair_wise_list = []
          pair = ()
          words = afterPreprocessQuery
          
          for token in words:
            if(token not in unigram_inverted_index):
              print(" positional invalid query")
              return
          
          if(len(words) == 1):
              documents = unigram_inverted_index[words[0]][1]
              print('Number of documents retrieved for query {} using positional inverted index:{}'.format(q,len(documents)))
              print('Number of documents retrieved for query {} using positional inverted index:'.format(q))
              for res_doc in documents:
                print(list_of_files[res_doc])
              return
              
        
          first_token =  words[0]
          result = unigram_inverted_index[first_token][1]
          for i in range(1,len(words)):
            token = unigram_inverted_index[words[i]][1]
            result = intersection(result,token)
          #print(result)
          if(len(result) == 0):
              print(" positional No document found")
              return
          

          
        
          for i, word in enumerate(words[:-1]):
            pair = (word, words[i+1])
            pair_wise_list.append(pair)
          temp = (words[0],words[-1])
          pair_wise_list.append(temp)

        
          result_documents = []
          count = 0
          for doc_id in result:
            for pair in pair_wise_list:
              if(pair == pair_wise_list[-1]):
                continue
              position_tk1 = positional_index[pair[0]][doc_id]
              position_tk2 = positional_index[pair[1]][doc_id]

              if(difference_within_k(position_tk1,position_tk2,1)):
                count = count + 1
            
            k = len(words) - 1
            pair = pair_wise_list[-1]
            #print(doc_id)
            if(difference_within_k(positional_index[pair[0]][doc_id],positional_index[pair[1]][doc_id],k)):
              count = count + 1
            # print(doc_id,"-->",count)
            if(count == len(pair_wise_list)-1 and len(words) == 2):
              result_documents.append(doc_id)
            if(count == len(pair_wise_list) and len(words)> 2):
              result_documents.append(doc_id)
            count = 0

          print('Number of documents retrieved for query {} using positional inverted index:{}'.format(q,len(result_documents)))
          print('Number of documents retrieved for query {} using positional inverted index:'.format(q))
          for res_doc in result_documents:
              print(list_of_files[res_doc])


In [ ]:
def search_query():
    # create set of stop words for preprocessing
    stopwords_set = set(stopwords.words('english'))
    
    # Get List of Files in Dataset
    #list_of_files = getListOfFiles('/content/drive/MyDrive/Information Retrieval/Assignment-1/Dataset_with_change/CSE508_Winter2023_Dataset/')
    
    # create dictizzonary of file with docID (integer) as key and full_path of file as value
    # file_dictionary = create_file_dictionary(list_of_files)
    
    # create positional index once and then load pickle file afterwards
    #Loading pre-processed files
    # positional_index = create_positional_index(newfiles)
    # pi_file = open('positional_index_pickle_file', 'rb')
    # positional_index = pickle.load(pi_file)
    # pi_file.close()
      
    N = int(input("input number of queries")) #input your query
    inputs = []
    for q in range(N):
        input_query = input("search your phrase queries")
        inputs.append(input_query)

    count = 1
    for query in inputs:
        search_bigram(query,count)
        print("----------xxxxxxx--------")
        search_positional(query,count)
        count = count +  1







In [ ]:
a = search_query()

input number of queries3
search your phrase querieswqdd  qd2j dqj di1hvqjh  vdhj wcj qhwvfjh fk
search your phrase queriestransition supersonic
search your phrase queriesexperimental transition
 bigram length of the input query to be <= 5
----------xxxxxxx--------
positional length of the input query to be <= 5
Number of documents retrieved for query 2 using bigram inverted index: 5
Number of documents retrieved for query 2 using bigram inverted index:
/content/CSE508_Winter2023_Dataset/cranfield0007
/content/CSE508_Winter2023_Dataset/cranfield0040
/content/CSE508_Winter2023_Dataset/cranfield0041
/content/CSE508_Winter2023_Dataset/cranfield0182
/content/CSE508_Winter2023_Dataset/cranfield1211
----------xxxxxxx--------
Number of documents retrieved for query 2 using positional inverted index:5
Number of documents retrieved for query 2 using positional inverted index:
/content/CSE508_Winter2023_Dataset/cranfield0007
/content/CSE508_Winter2023_Dataset/cranfield0040
/content/CSE508_Winter2

In [ ]:
sstrapositional_index['transition']

{6: [6, 28, 48, 69, 124],
 7: [8, 12, 35, 59],
 8: [0, 12, 36, 102, 107],
 23: [86],
 39: [3, 24, 53, 67, 75],
 40: [0, 6, 33],
 42: [7, 13, 42, 77],
 52: [0, 18, 61, 122],
 78: [6, 21, 37, 53, 65, 84, 95],
 79: [8, 32, 53, 113, 143],
 88: [76],
 93: [228],
 95: [5, 19, 31, 71, 93, 110, 127, 135],
 122: [42, 121],
 124: [84, 94, 108],
 132: [87],
 141: [62],
 170: [75],
 181: [2, 24, 39],
 186: [8, 62, 75, 113],
 198: [37, 62, 71, 139, 187],
 206: [4, 21, 48, 56],
 211: [104, 112],
 243: [112, 187],
 260: [48],
 271: [16,
  23,
  33,
  52,
  65,
  73,
  79,
  84,
  108,
  144,
  152,
  162,
  175,
  189,
  198,
  228,
  231,
  237,
  247],
 292: [6, 15, 31, 44],
 293: [140],
 313: [10, 27, 98],
 314: [8, 51, 70, 102, 125, 136, 163, 212, 236, 244, 259],
 329: [53],
 336: [2, 30],
 337: [44, 67],
 340: [48, 108],
 343: [91],
 345: [21, 62],
 417: [0, 9, 33],
 439: [39],
 440: [106],
 503: [89, 96],
 504: [0, 13, 36, 38],
 521: [178, 184, 191],
 524: [39],
 525: [43],
 534: [35],
 535: [0